In [ ]:
import pandas
import urllib 
import datetime
import time
import random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

xls_file = pandas.ExcelFile('IPO-Last100.xlsx')
df = xls_file.parse('Sheet1')
ipoDataFrame=df[["Company","Symbol","Industry","Offer Date"]]
ipoDataFrame=ipoDataFrame.sort_values(by='Offer Date', ascending=0)

twitterfilter1='"${}" since:{:%Y-%m-%d} until:{:%Y-%m-%d}'
twitterfilter2='"{}" since:{:%Y-%m-%d} until:{:%Y-%m-%d}'

symbolAndUrlDict=dict.fromkeys(list(ipoDataFrame['Symbol']), [])
browser=webdriver.Chrome()
timetosleep=random.randrange(5, 100)
time.sleep(timetosleep)
print("start")

numberOfTweetsBefore=0
numberOfTweetsAfter=0
numberOfTryingToPageDown=3

for key, value in symbolAndUrlDict.items():

    currentCompany=ipoDataFrame.loc[ipoDataFrame['Symbol'] == key]
    offerDate=currentCompany['Offer Date'].iloc[0].date()
    fivedaysBeforeOfferDate=offerDate-datetime.timedelta(days=6)
    oneMonthAfterOfferDate=offerDate+datetime.timedelta(days=35)
    companyName=currentCompany['Company'].iloc[0]
    companySymbol=key
    print(companySymbol)
    queryString1= twitterfilter1.format(companySymbol,fivedaysBeforeOfferDate, oneMonthAfterOfferDate)
    queryStringDictionary1={'l':'','q':queryString1,'src':'typd'}
    
    queryString2= twitterfilter2.format(companyName,fivedaysBeforeOfferDate, oneMonthAfterOfferDate)
    queryStringDictionary2={'l':'','q':queryString2,'src':'typd'}
  
    url1=urllib.parse.urlencode(queryStringDictionary1)
    url2=urllib.parse.urlencode(queryStringDictionary2)
    url1='https://twitter.com/search?'+url1.replace('+', '%20')
    url2='https://twitter.com/search?'+url2.replace('+', '%20')
    
    for i in range(0,2):
        timetosleep=random.randrange(5, 27)
        print('time to sleep:',timetosleep)
        time.sleep(timetosleep)
        if(i==0):
            print("Browse url 1 ")
            browser.get(url1)
        else:
            print("Browse url 1 ")
            browser.get(url2)
        body=browser.find_element_by_tag_name('body')
        numberOfTweetsBefore=0
        numberOfTweetsAfter=0
        numberOfTryingToPageDown=0
        numberOfTryingToPageDown2=0
        for k in range(random.randrange(5,20)):
            body.send_keys(Keys.PAGE_DOWN)
            browser.execute_script("window.scrollBy(0,250);")
            time.sleep(0.5)
            countTryPageDown=0
        tweets=browser.find_elements_by_class_name('tweet-text')
        numberOfTweetsBefore=len(tweets)
        numberOfTweetsAfter=len(tweets)
        while ( (numberOfTweetsBefore != numberOfTweetsAfter and numberOfTryingToPageDown2 < 500) or (numberOfTweetsBefore == numberOfTweetsAfter and numberOfTryingToPageDown<6)):
            if(numberOfTweetsBefore == numberOfTweetsAfter):
                numberOfTryingToPageDown=numberOfTryingToPageDown+1
            else:
               numberOfTryingToPageDown2=numberOfTryingToPageDown2+1
            for j in range(random.randrange(5,15)):
                numberOfTweetsBefore=len(tweets)
                body.send_keys(Keys.PAGE_DOWN)
                #timetosleep=random.randrange(1,5)
                time.sleep(1)

            print("numberOfTweetsAfter : ",numberOfTweetsAfter)
            print("numberOfTweetsBefore : ",numberOfTweetsBefore)
            tweets=browser.find_elements_by_class_name('tweet-text')
            numberOfTweetsAfter=len(tweets)
            print("numberOfTweetsAfter : ",numberOfTweetsAfter)
            print("numberOfTweetsBefore : ",numberOfTweetsBefore)
            timetosleep=random.randrange(5, 10)
            print("sleep for ",timetosleep)
            time.sleep(timetosleep)
            if(i==0):
                file = open(companySymbol+'.html','w')
            else:
                file = open(companySymbol+'_2.html','w')
            print("write to file")
            file.write(browser.page_source)
            file.close()   
        if(i==0):
            file = open(companySymbol+'.html','w')
        else:
            file = open(companySymbol+'_2.html','w')
        file.write(browser.page_source)
        file.close()   